In [1]:
from IPython.display import display, HTML
import json
import os
import numpy
from tqdm import tqdm
import warnings
import pandas as pd
import json
import torch

Innovations: new BERT, new dataset, added query expansion for the BM25 while not using the expanded query (The reasoning is beacause it reduces performamce on BERT according to the cited paper.)

In [2]:
path = "documents/reformatted_data.jsonl"
df = pd.read_json(path, lines=True)
df

,id,contents
0,2212463,Patient Information:\nAge: 24 years\nGender: F...
1,2218808,Patient Information:\nAge: 30 years old\nGende...
2,2224301,Patient Information:\nAge: 43 years\nGender: M...
3,2232999,Medical Record Description:\nPatient Informati...
4,2254405,Medical Record Description:\nPatient Informati...
...,...,...
357,5271378,Patient Information:\nAge: 58 years\nGender: M...
358,5278489,Patient Information:\nAge: 48 years\nGender: F...
359,5286851,Medical Record Description:\nPatient Informati...
360,5288592,Patient Information:\nAge: 1 day old\nGender: ...


In [3]:
# SETTING JAVA_HOME
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-21-openjdk-amd64'  # Adjust this path as needed

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
model = AutoModelForMaskedLM.from_pretrained("medicalai/ClinicalBERT")

/home/darryl-chandra/Documents/information-retrieval/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import subprocess
import os

# Check current Java version
try:
    result = subprocess.run(['java', '-version'], capture_output=True, text=True)
    print("Current Java version:")
    print(result.stderr)
    
    # Also check JAVA_HOME
    java_home = os.environ.get('JAVA_HOME', 'Not set')
    print(f"\nJAVA_HOME: {java_home}")
except FileNotFoundError:
    print("Java not found in PATH")

Current Java version:
openjdk version "21.0.8" 2025-07-15
OpenJDK Runtime Environment (build 21.0.8+9-Ubuntu-0ubuntu124.04.1)
OpenJDK 64-Bit Server VM (build 21.0.8+9-Ubuntu-0ubuntu124.04.1, mixed mode, sharing)


JAVA_HOME: /usr/lib/jvm/java-21-openjdk-amd64


https://arxiv.org/pdf/2103.05256 --consideration why the expanded index is not used.

In [24]:
class DocumentRetriever:
    def __init__(self, index_path):
        from pyserini.search.lucene import LuceneSearcher
        self.searcher = LuceneSearcher(index_path)
        # === The line below is one of our innovation to the pipeline. It enables RM3 relevance feedback ===
        self.searcher.set_rm3(fb_terms=5, fb_docs=10, original_query_weight=0.7)     
    def extract_bert(self, text, tokenizer, model):
        text_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
        text_words = tokenizer.convert_ids_to_tokens(text_ids[0])[1:-1]

        n_chunks = int(numpy.ceil(float(text_ids.size(1))/510))
        states = []
        
        for ci in range(n_chunks):
            text_ids_ = text_ids[0, 1+ci*510:1+(ci+1)*510]            
            text_ids_ = torch.cat([text_ids[0, 0].unsqueeze(0), text_ids_])
            if text_ids[0, -1] != text_ids[0, -1]:
                text_ids_ = torch.cat([text_ids_, text_ids[0,-1].unsqueeze(0)])
            
            with torch.no_grad():
                state = model(text_ids_.unsqueeze(0))[0]
                state = state[:, 1:-1, :]
            states.append(state)

        state = torch.cat(states, axis=1)
        return text_ids, text_words, state[0]
    def cross_match(self, state1, state2):
        state1 = state1 / torch.sqrt((state1 ** 2).sum(1, keepdims=True))
        state2 = state2 / torch.sqrt((state2 ** 2).sum(1, keepdims=True))
        sim = (state1.unsqueeze(1) * state2.unsqueeze(0)).sum(-1)
        return sim
    def search(self, query, k=10):
        hits = self.searcher.search(query, k)
        paragraph_states = []
        for i in range(len(hits)):
            doc = hits[i]
            doc_json = json.loads(doc.lucene_document.get('raw'))
            doc_ids, doc_words, doc_state = self.extract_bert(doc_json['contents'], tokenizer, model)
            paragraph_states.append(doc_state)
            # doc_ids, doc_words, doc_state = extract_scibert(doc_json['contents'], tokenizer, model) --- IGNORE ---
            query_ids, query_words, query_state = self.extract_bert(query, tokenizer, model)
            sim_matrices = []
            for i in range(len(paragraph_states)):
                sim = self.cross_match(query_state, paragraph_states[i])
                sim_matrices.append(sim)
            paragraph_relevance = [torch.max(sim).item() for sim in sim_matrices]
            rel_index = numpy.argsort(paragraph_relevance)[-5:][::-1]
            # Print top 5 relevant documents
            for idx in rel_index:
                print(json.loads(hits[idx].lucene_document.get('raw'))['contents'])

In [ ]:
retriever = DocumentRetriever('indexes/sample_collection_jsonl')
query = "What is the treatment for COVID-19?"
retriever.search(query, k=10)

Patient Information:
Age: 30 years
Gender: Female
Chief Complaint: Rectal bleeding during defecation for over 2 months.
Present Illness: The patient reports intermittent rectal bleeding during bowel movements for the past 2 months. The blood is bright red, low to moderate in volume. She seeks further diagnosis and treatment.
Past Medical History: Denies a history of hypertension, diabetes, heart disease, and drug or food allergies, as well as any history of surgeries or trauma. No recent travel history to Wuhan, Beijing, or abroad in the past 2 weeks. No contact with returnees from Wuhan or abroad, nor with asymptomatic carriers or suspected or confirmed cases of COVID-19. There are no reported COVID-19 infections in her residential community, and there have been no family clusters of illness.
Physical Examination: Rectal examination shows external hemorrhoids at the 11-12 o'clock position. No significant masses detected on digital rectal examination. Anoscopy reveals internal hemorrho